In [2]:
import os
import glob
import cv2

# Define the paths to the folders containing the images
eval_path = os.path.abspath('C:/Users/wdmoreno/OneDrive - Asesoftware S.A.S/Documentos/Food_data/evaluation')
train_path = os.path.abspath('C:/Users/wdmoreno/OneDrive - Asesoftware S.A.S/Documentos/Food_data/training')
val_path = os.path.abspath('C:/Users/wdmoreno/OneDrive - Asesoftware S.A.S/Documentos/Food_data/validation')

# Define a list of folder names
folders = ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles-Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable-Fruit']

# Initialize arrays to store images from each directory
eval_images = []
eval_labels = []
train_images = []
train_labels = []
val_images = []
val_labels = []

# Loop over the folders
for folder in folders:
    # Read images from evaluation directory
    eval_folder = os.path.join(eval_path, folder)
    eval_images.extend([cv2.imread(img_path) for img_path in glob.glob(os.path.join(eval_folder, '*.jpg'))])
    eval_labels.extend([folder] * len(os.listdir(eval_folder)))  # add labels for each image

    # Read images from training directory
    train_folder = os.path.join(train_path, folder)
    train_images.extend([cv2.imread(img_path) for img_path in glob.glob(os.path.join(train_folder, '*.jpg'))])
    train_labels.extend([folder]* len(os.listdir(train_folder))) 

    # Read images from validation directory
    val_folder = os.path.join(val_path, folder)
    val_images.extend([cv2.imread(img_path) for img_path in glob.glob(os.path.join(val_folder, '*.jpg'))])
    val_labels.extend([folder] * len(os.listdir(val_folder)))  # add labels for each image

# Print the number of images in each directory
print(f'Number of evaluation images: {len(eval_images)}')
print(f'Number of training images: {len(train_images)}')
print(f'Number of validation images: {len(val_images)}')

print(f'Number of evaluation labels: {len(eval_labels)}')
print(f'Number of training labels: {len(train_labels)}')
print(f'Number of validation labels: {len(val_labels)}')

Number of evaluation images: 3347
Number of training images: 9860
Number of validation images: 3430
Number of evaluation labels: 3347
Number of training labels: 9860
Number of validation labels: 3430


### Con OpenCV

In [8]:
from PIL import Image
import statistics
import os
import cv2
import glob

# Definir la carpeta de salida para las imágenes comprimidas
output_folder = 'carpeta_de_salida_openCV'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Definir el número de imágenes a comprimir
num_images = 100
compresion_array=[]

# Iterar a través de las primeras num_images imágenes y comprimir
for i, img_path in enumerate(glob.glob(os.path.join(train_folder, '*.jpg'))[:num_images]):
    # Cargar la imagen original
    original_img = Image.open(img_path)
    original_size = os.path.getsize(img_path)
    
    # Comprimir la imagen y guardarla en el directorio de salida
    output_filepath = os.path.join(output_folder, f'image_{i}.jpg')
    img = cv2.imread(img_path)
    cv2.imwrite(output_filepath, img, [cv2.IMWRITE_JPEG_QUALITY, 60])
    
    # Calcular el tamaño de la imagen comprimida
    compressed_size = os.path.getsize(output_filepath)
    
    # Calcular el porcentaje de compresión
    compression_ratio = compressed_size / original_size
    compression_percentage = (1 - compression_ratio) * 100
    compresion_array.append(compression_percentage)
    
    print(f"La imagen {i} se ha comprimido y es un {compression_percentage}% de la original")
    
# calcular el promedio de la lista compresion_array
mean_compression = statistics.mean(compresion_array)

print(f"El promedio de compresión es {mean_compression}%")


La imagen 0 se ha comprimido y es un 81.27921749151237% de la original
La imagen 1 se ha comprimido y es un 52.402726770840836% de la original
La imagen 2 se ha comprimido y es un 84.16523122424849% de la original
La imagen 3 se ha comprimido y es un 45.2256702386169% de la original
La imagen 4 se ha comprimido y es un 61.52637009749559% de la original
La imagen 5 se ha comprimido y es un 42.670194621993396% de la original
La imagen 6 se ha comprimido y es un 60.15019061390825% de la original
La imagen 7 se ha comprimido y es un 43.739101085672495% de la original
La imagen 8 se ha comprimido y es un 43.86692668738018% de la original
La imagen 9 se ha comprimido y es un 47.465844982667605% de la original
La imagen 10 se ha comprimido y es un 46.79136880685476% de la original
La imagen 11 se ha comprimido y es un 69.14916749502981% de la original
La imagen 12 se ha comprimido y es un 43.91788181120491% de la original
La imagen 13 se ha comprimido y es un 73.0683897557526% de la original


### Con SVD

In [9]:
import os
import glob
import numpy as np
from PIL import Image
import statistics

# Definir la carpeta de salida para las imágenes comprimidas
output_folder = 'carpeta_de_salida_SVD'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Definir el número de imágenes a comprimir
num_images = 100
num_components = 100

# Lista para almacenar los porcentajes de compresión de cada imagen
compresion_array=[]

# Iterar a través de las primeras num_images imágenes y comprimir
for i, img_path in enumerate(glob.glob(os.path.join(train_folder, '*.jpg'))[:num_images]):
    # Cargar la imagen original
    original_img = Image.open(img_path)
    original_size = os.path.getsize(img_path)
    
    # Comprimir la imagen y guardarla en el directorio de salida
    output_filepath = os.path.join(output_folder, f'image_{i}.jpg')
    img = Image.open(img_path).convert('RGB')  # Convertir la imagen a RGB
    img_array = np.asarray(img, dtype=np.float32) / 255.0
    U, S, Vt = np.linalg.svd(img_array, full_matrices=False)
    compressed_img_array = np.dot(U[:, :num_components], np.dot(np.diag(S[:num_components]), Vt[:num_components, :].T))
    compressed_img_array = np.clip(compressed_img_array, 0, 1)
    compressed_img = Image.fromarray(np.uint8(compressed_img_array * 255), mode='RGB')
    compressed_img.save(output_filepath)

    
    # Calcular el tamaño de la imagen comprimida
    compressed_size = os.path.getsize(output_filepath)
    
    # Calcular el porcentaje de compresión
    compression_ratio = compressed_size / original_size
    compression_percentage = (1 - compression_ratio) * 100
    compresion_array.append(compression_percentage)
    
    print(f"La imagen {i} se ha comprimido y es un {compression_percentage}% de la original")
    
# calcular el promedio de la lista compresion_array
mean_compression = statistics.mean(compresion_array)

print(f"El promedio de compresión es {mean_compression}%")



La imagen 0 se ha comprimido y es un 96.92612575674274% de la original
La imagen 1 se ha comprimido y es un 97.00439259739323% de la original
La imagen 2 se ha comprimido y es un 94.07423021312614% de la original
La imagen 3 se ha comprimido y es un 83.16336692825558% de la original
La imagen 4 se ha comprimido y es un 92.12031206302979% de la original
La imagen 5 se ha comprimido y es un 75.96314774375568% de la original
La imagen 6 se ha comprimido y es un 83.90627054029184% de la original
La imagen 7 se ha comprimido y es un 72.75412049277156% de la original
La imagen 8 se ha comprimido y es un 75.49910755602565% de la original
La imagen 9 se ha comprimido y es un 82.64041507125542% de la original
La imagen 10 se ha comprimido y es un 77.2845090885077% de la original
La imagen 11 se ha comprimido y es un 87.84402957256461% de la original
La imagen 12 se ha comprimido y es un 84.72947045280122% de la original
La imagen 13 se ha comprimido y es un 95.0224660943017% de la original
La i